In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
import os
from IPython.display import display_html
import marko

from taogpt.orchestrator import *
from taogpt.utils import *
from taogpt.llm_model import LangChainLLM
from taogpt.prompts import PromptDb
import taogpt.utils as utils

utils.enable_debugging(0)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
TEMPERATURE = 0.7

In [4]:
with open(os.path.join(os.environ['HOME'], '.ssh', 'openai-zillow.json'), 'r') as f:
    credentials = json.load(f)
os.environ["OPENAI_API_KEY"] = credentials['key']
os.environ["OPENAI_API_BASE"] = credentials['url']
llm3_5 = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=TEMPERATURE)
llm4 = ChatOpenAI(model_name='gpt-4-32k', temperature=TEMPERATURE)

conversation = ConversationChain(llm=llm3_5)
conversation.predict(input="What's your model version?")

"I am using OpenAI's GPT-3 model, which is the most advanced version available."

In [5]:
llm3_5.model_name, llm4.model_name

('gpt-3.5-turbo-16k', 'gpt-4-32k')

In [ ]:
prompts = PromptDb.load_defaults()
logger = MarkdownLogger('logs/taogpt_log.md')
config = Config(
    ask_user_before_execute_codes=False,
    initial_expansion=1,
    max_tree_branches=2,
    max_tokens=10000,
    check_final=True
)
executor = Orchestrator(
    config=config,
    llm=LangChainLLM(llm4, logger=logger),
    prompts=prompts,
    markdown_logger=logger,
    # smarter_llm=LangChainLLM(llm4, logger=logger),
)

experiment_name = 'example'
executor.start("""
I want to travel to Moscow. Find me a travel route.
""", analyze_first=True)


In [7]:
# executor.resume(10000)

In [11]:
logger = MarkdownLogger(f'examples/{experiment_name}.final.md')
logger.log_conversation(executor.show_conversation_thread(with_extras=True))
logger.log(f"**total tokens**: {executor.llm.total_tokens}")

In [ ]:
with open(logger._log_path, 'r') as f:
    html_output = marko.convert(f.read())
    display_html(html_output, raw=True)
